In [20]:
from braindecode.datasets import MOABBDataset
from braindecode.preprocessing import preprocess, Preprocessor
dataset = MOABBDataset(dataset_name="BNCI2014_001", subject_ids=[1,2,3,4,5,6,7,8,9])

In [33]:
preprocessors = [
    Preprocessor('pick_types', eeg=True, meg=False, stim=True),
    Preprocessor('resample', sfreq=100)
]
# Preprocess the data
preprocess(dataset, preprocessors, n_jobs=-1)

In [34]:
from braindecode.preprocessing import create_windows_from_events

trial_start_offset_seconds = -0.5
# Extract sampling frequency, check that they are same in all datasets
sfreq = dataset.datasets[0].raw.info["sfreq"]
assert all([ds.raw.info["sfreq"] == sfreq for ds in dataset.datasets])
# Calculate the window start offset in samples.
trial_start_offset_samples = int(trial_start_offset_seconds * sfreq)

# Create windows using braindecode function for this. It needs parameters to
# define how windows should be used.
windows_dataset = create_windows_from_events(
    dataset,
    trial_start_offset_samples=trial_start_offset_samples,
    trial_stop_offset_samples=0,
    preload=True,
)

Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']

In [37]:
splitted = windows_dataset.split("session")
train_set = splitted['0train']  # Session train
test_set = splitted['1test']  # Session evaluation

In [38]:
import torch
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet

n_classes = 4
classes = list(range(n_classes))
# Extract number of chans and time steps from dataset
n_channels = windows_dataset[0][0].shape[0]
input_window_samples = windows_dataset[0][0].shape[1]

model = ShallowFBCSPNet(
    n_channels,
    n_classes,
    n_times=input_window_samples,
    final_conv_length="auto",
    add_log_softmax=False,
)

# Display torchinfo table describing the model
print(model)


Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
ShallowFBCSPNet (ShallowFBCSPNet)        [1, 23, 450]              [1, 4]                    --                        --
├─Ensure4d (ensuredims): 1-1             [1, 23, 450]              [1, 23, 450, 1]           --                        --
├─Rearrange (dimshuffle): 1-2            [1, 23, 450, 1]           [1, 1, 450, 23]           --                        --
├─CombinedConv (conv_time_spat): 1-3     [1, 1, 450, 23]           [1, 40, 426, 1]           37,840                    --
├─BatchNorm2d (bnorm): 1-4               [1, 40, 426, 1]           [1, 40, 426, 1]           80                        --
├─Expression (conv_nonlin_exp): 1-5      [1, 40, 426, 1]           [1, 40, 426, 1]           --                        --
├─AvgPool2d (pool): 1-6                  [1, 40, 426, 1]           [1, 40, 24, 1]            --                        [75, 1]
├─Express

In [41]:
from skorch.callbacks import LRScheduler

from braindecode import EEGClassifier

lr = 0.0625 * 0.01
weight_decay = 0
batch_size = 64
n_epochs = 5

clf = EEGClassifier(
    model,
    criterion=torch.nn.NLLLoss,
    optimizer=torch.optim.AdamW,
    train_split=None,
    optimizer__lr=lr,
    optimizer__weight_decay=weight_decay,
    batch_size=batch_size,
    callbacks=[
        "accuracy",
        ("lr_scheduler", LRScheduler("CosineAnnealingLR", T_max=n_epochs - 1)),
    ],
    classes=classes,
    max_epochs=n_epochs,
)
# Model training for a specified number of epochs. `y` is None as it is already supplied
# in the dataset.
clf.fit(train_set, y=None)

# evaluated the model after training
y_test = test_set.get_metadata().target
test_acc = clf.score(test_set, y=y_test)
print(f"Test acc: {(test_acc * 100):.2f}%")

  epoch    train_accuracy    train_loss      lr     dur
-------  ----------------  ------------  ------  ------
      1            0.7500    -3973.2818  0.0006  5.3093
      2            0.6042    -4310.7288  0.0005  4.9990
      3            0.5602    -4502.0030  0.0003  4.9337
      4            0.5440    -4604.9445  0.0001  4.8331
      5            0.5000    -4630.8689  0.0000  4.9390
Test acc: 50.00%


In [48]:
predicted = clf.predict(test_set)
print(predicted)

[1 0 0 ... 3 1 3]
